<a href="https://colab.research.google.com/github/TirilaPatricGabriel/ML/blob/main/AnnUsingPyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.parallel

dataset = pd.read_csv("Churn_Modelling.csv")
X = np.array(dataset.iloc[:, 3:-1])
y = np.array(dataset.iloc[:, -1])

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(), [1])], remainder="passthrough")
X = ct.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

input_dim = X_train.shape[1]
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

class ANN(nn.Module):
  def __init__(self, input_dim):
    super(ANN, self).__init__()
    self.dense1 = nn.Linear(input_dim, 256)
    self.dense2 = nn.Linear(256, 128)
    self.dense3 = nn.Linear(128, 2)
    self.activation = nn.ReLU()

  def forward(self, x):
    x = self.activation(self.dense1(x))
    x = self.activation(self.dense2(x))
    x = self.dense3(x)
    return x
ann = ANN(input_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ann.parameters(), lr=0.01)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

n_ep = 101
for epoch in range(n_ep):
  ann.train()
  epoch_loss = 0

  for batch_X, batch_y in train_loader:
    optimizer.zero_grad()
    output = ann(batch_X)

    loss = criterion(output, batch_y)
    epoch_loss += loss.item()

    loss.backward()
    optimizer.step()

  print(f"epoch: {epoch} with loss: {epoch_loss}")

ann.eval()
correct = 0
total = 0

with torch.no_grad():
  for batch_X, batch_y in test_loader:
    output = ann(batch_X)
    predictions = torch.argmax(output, dim=1)
    correct = (predictions == batch_y).sum().item()
    total = batch_y.size(0)

accuracy = correct / total
print(accuracy)